# Machine Learning for Page Sage

This notebook should serve as a tutorial and explanatory process for the Page Sage recommendation algorithm.

First, let's create some preprocessors for the data since we know what format to expect them.

----

In [189]:
import requests
import os
import re
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import make_pipeline

from copy import deepcopy


class BookClassifier(object):
    def __init__(self, volumes=[], ratings=[]):
        if (len(volumes) == 0) or (len(ratings) == 0):
            raise ValueError('Initial values cannot be zero')
        if (len(volumes) != len(ratings)):
            raise ValueError('Labels and data must have the same length')

        self.volumes = volumes
        self.ratings = ratings
        self.__book_data = []
        self.categories = dict()
        self.maturities = dict()
        self.books = []
        self.X_train, self.y_train = [],[]
        self.__model = None
        self.set_up()


    def set_up(self):
        self.__book_data = self.__pull_books(self.volumes, self.ratings)
        self.__find_categories(self.__book_data)
        #self.maturities = self.__find_maturities(self.book_data)
        self.books = self.__reformat(self.__book_data)
        self.X_train, self.y_train = self.__x_y_train(self.books)
        ##self.scikit_kpca = KernelPCA(n_components=3, kernel='rbf')
        ##self.X_train = self.scikit_kpca.fit_transform(self.X_train)


    def __pull_books(self, volumes, labels=[]):
        book_data = []
        search_key = str(os.environ.get('SEARCH_KEY'))
        baseURL = 'https://www.googleapis.com/books/v1/volumes/'
        endURL = '?key=' + search_key

        headers = {'Accept': 'application/json'}
        if type(volumes) == type(str()):
            volumes = [volumes]
        
        for index, volume in enumerate(volumes):
            url = baseURL + str(volume) + endURL

            resp = requests.get(url, params=headers)
            if not resp.ok:
                raise ValueError('Response error; could not make call')
            book_info = resp.json()
            new_book = {}

            try:
                page_count  =  int(book_info['volumeInfo']['pageCount'])
            except (KeyError):
                page_count = 100

            try:
                categories = self.__category_preprocessor(book_info['volumeInfo']['categories'])
            except (KeyError):
                categories = self.__category_preprocessor(['Fiction'])

            try:
                average_rating = float(book_info['volumeInfo']['averageRating'])
            except (KeyError):
                average_rating = float(3)

            try:
                ratings_count = int(book_info['volumeInfo']['ratingsCount'])
            except (KeyError):
                ratings_count = int(0)

            '''
            try:
                maturity_rating = book_info['volumeInfo']['maturityRating']
            except (KeyError):
                maturity_rating = 'NOT_MATURE'
            '''

            if labels != []:
                book_data.append({
                    'user_rating'       : labels[index],
                    'page_count'        : page_count,
                    'categories'        : categories,
                    'average_rating'    : average_rating,
                    'ratings_count'     : ratings_count
                    #'maturity_rating'   : maturity_rating
                })
            else:
                book_data.append({
                    'page_count'        : page_count,
                    'categories'        : categories,
                    'average_rating'    : average_rating,
                    'ratings_count'     : ratings_count
                    #'maturity_rating'   : maturity_rating
                })

        return book_data


    def __category_preprocessor(self, categories):
        if type(categories) != type(list()):
            return categories
        
        cats = []
        for category in categories:
            cat_split = category.split('/')
            for cat in cat_split:
                cat = cat.strip().lower()
                if cat not in cats:
                    cats.append(cat)

        return cats


    def __find_categories(self, books):
        index = 2
        for book in books:
            for category in book['categories']:
                if category not in self.categories.keys():
                    self.categories[category] = index
                    index += 1


    def __find_maturities(self, books):
        index = 0
        for book in books:
            if book['maturity_rating'] not in self.maturities:
                self.maturities[book['maturity_rating']] = index
                index += 1


    def __reformat(self, books):
        new_books = []
        for book in books:
            new_book = []
            for data in book:
                if 'categories' == data:
                    for category in self.categories:
                        new_book.append(0)
                elif 'maturity_rating' == data:
                    for maturity in self.maturities:
                        new_book.append(0)
                else:
                    new_book.append(book[data])
            new_books.append(new_book)
        for index, book in enumerate(new_books):
            for category in self.categories:
                if category in books[index]['categories']:
                    new_books[index][self.categories[category]] = books[index]['categories'].count(category)
            '''for maturities in self.maturities:
                if maturity in books[index]['maturities']:
                    new_books[index][len(maturities[maturity])
            '''
        return new_books

    def __reformat_volume(self, book):
        new_book=[]
        for data in book:
            if 'categories' == data:
                for category in self.categories:
                    new_book.append(0)
            else:
                new_book.append(book[data])
        for category in self.categories:
            if category in book['categories']:
                new_book[self.categories[category]] = book['categories'].count(category)
        return new_book
            

    def __x_y_train(self, books):
        books = deepcopy(books)
        labels = []
        new_books = []

        for book in books:
            item = deepcopy(book)
            #label = np.ndarray(1)
            #label[0] = int(item.pop(0))
            #labels.append(label)
            labels.append(int(item.pop(0)))
            new_books.append(item)

        return (np.array(new_books), np.array(labels))


    def fit(self):
        '''
        model = keras.models.Sequential()
        model.add(keras.layers.Dense(units=20, input_dim=self.X_train.shape[1],
                                     kernel_initializer='glorot_uniform',
                                     bias_initializer='zeros',
                                     activation='relu'))

        model.add(keras.layers.Dense(units=20, input_dim=self.X_train.shape[1],
                                     kernel_initializer='glorot_uniform',
                                     bias_initializer='zeros',
                                     activation='relu'))

        model.add(keras.layers.Dense(units=self.y_train.shape[1], input_dim=self.X_train.shape[1],
                                     kernel_initializer='glorot_uniform',
                                     bias_initializer='zeros',
                                     activation='softmax'))

        sgd_optimizer = keras.optimizers.SGD(lr=0.001, decay=1e-7, momentum=0.9)

        model.compile(optimizer='adadelta', loss='binary_crossentropy')

        history = model.fit(self.X_train, self.y_train,
                            batch_size=3, epochs=15, verbose=1,
                            validation_split=0.1)

        self.y_train_pred = model.predict_classes(self.X_train, verbose=0)

        '''
        
        pipe_lr = make_pipeline(StandardScaler(),
                                PCA(),
                                LogisticRegression(random_state=1, solver='lbfgs'))
        
        #self.y_train = 
        pipe_lr.fit(self.X_train, self.y_train)
        
        self.y_train_pred = pipe_lr.predict(self.X_train)
        
        self.__model = pipe_lr
        
        #pipe_sgd = make_pipeline(StandardScaler(), PCA(), SGDClassifier(max_iter=1000, tol=1e-3))
        #pipe_sgd.fit(self.X_train, self.y_train)
        #self.y_train_pred = pipe_sgd.predict(self.X_train)
        self.__model = pipe_lr
    
        
    def feature_space(self):
        return str(self.X_train.shape[0]) + ', ' + str(self.X_train.shape[1])
    
    
    def train_acc(self):
        return np.sum(self.y_train_pred == self.y_train, axis=0) / self.y_train.shape[0]
    
        
    def __preprocess_book(self, book):
        new_book = deepcopy(book)
        book_data = self.__pull_books(new_book)
        
        if type(book_data[0]) == type(dict()):
            return np.asarray(self.__reformat_volume(book_data[0]))
        raise TypeError("Goddamn")
    
    def sample_data(self):
        return 'Original:\n' + str(self.__book_data[0]) + '\nPost-processed:\n' + str(self.X_train)
    
    
    def predict(self, book=[]):
        if book == [] or book =="":
            raise ValueError('Parameter cannot be an empty book')
        if self.__model == None:
            raise ValueError('Classifier needs training before predictions')
        if type(book) != type(str()):
            raise ValueError('Book must a valid volumeID of type "String"')

        X_sample = self.__preprocess_book(book)
        
        return (self.__model.predict([X_sample]), np.max(self.__model.predict_proba([X_sample])))


In [190]:
test_volumes = []
test_labels = []

with open('emily_books_2_tier.txt', 'r') as test_file:
    for line in test_file:
        items = line.split(',')
        test_volumes.append(items[0])
        test_labels.append(int(items[1].strip()))

print('Length of volumes:\t', len(test_volumes))
print('Length of labels:\t', len(test_labels))

Length of volumes:	 61
Length of labels:	 61


In [191]:
test_model = BookClassifier(test_volumes, test_labels)

In [193]:
backup_model = deepcopy(test_model)

In [194]:
backup_model.fit()

In [195]:
print('Categories', backup_model.categories)

Categories {'fiction': 2, 'thrillers': 3, 'suspense': 4, 'fantasy': 5, 'contemporary': 6, 'psychological': 7, 'young adult fiction': 8, 'science fiction': 9, 'space opera': 10, 'romance': 11, 'general': 12, 'occult & supernatural': 13, 'historical': 14, 'paranormal': 15, 'fairy tales, folk tales, legends & mythology': 16, 'action & adventure': 17, 'juvenile fiction': 18, 'wizards & witches': 19, 'school & education': 20, 'boarding school & prep school': 21, 'fantasy & magic': 22, 'history': 23, 'united states': 24, 'nature': 25, 'animals': 26, 'horses': 27, 'horror': 28, 'superheroes': 29, 'gaslamp': 30, 'literary': 31, 'family life': 32, 'coming of age': 33, 'epic': 34, 'dark fantasy': 35, 'classics': 36, 'media tie-in': 37, 'mystery & detective': 38, 'humorous': 39, 'absurdist': 40, 'dragons & mythical creatures': 41, 'romantic comedy': 42, 'women': 43, 'frankenstein (fictitious character)': 44, "frankenstein's monster (fictitious character)": 45, 'drama': 46, 'frankenstein, victor (

In [196]:
print(backup_model.sample_data())

Original:
{'user_rating': 1, 'page_count': 352, 'categories': ['fiction', 'thrillers', 'suspense', 'fantasy', 'contemporary', 'psychological'], 'average_rating': 4.0, 'ratings_count': 34}
Post-processed:
[[3.520e+02 1.000e+00 1.000e+00 ... 0.000e+00 4.000e+00 3.400e+01]
 [6.080e+02 0.000e+00 0.000e+00 ... 0.000e+00 4.500e+00 5.700e+01]
 [5.920e+02 1.000e+00 1.000e+00 ... 0.000e+00 3.500e+00 6.790e+02]
 ...
 [4.850e+02 0.000e+00 0.000e+00 ... 1.000e+00 4.000e+00 2.473e+03]
 [3.840e+02 1.000e+00 1.000e+00 ... 0.000e+00 4.000e+00 1.540e+02]
 [3.080e+02 1.000e+00 0.000e+00 ... 0.000e+00 3.000e+00 0.000e+00]]


In [197]:
print('Train Accuracy: %.2f%%' % (100*backup_model.train_acc()))

Train Accuracy: 95.08%


In [199]:
aConjuringOfLight = 'bP60vwEACAAJ'
backup_model.predict(aConjuringOfLight)

(array([1]), 0.7302507243407482)

Factory method for classifier

In [182]:
def create_classifier(user_ratings=[], user_volumes=[]):
    if user_ratings == [] or user_volumes == []:
        raise TypeError('User ratings')
    model = BookClassifier(user_ratings, user_volumes)
    model.fit()
    return deepcopy(model)

In [200]:
TortillaCurtain = 'yYPnwMpmeT4C'
backup_model.predict(TortillaCurtain)

(array([1]), 0.9234770106916396)

In [203]:
nineteeneightyfour = 'VO8nDwAAQBAJ'
backup_model.predict(nineteeneightyfour)

(array([1]), 0.9762450897058621)

In [204]:
TheStand = 'UbfnTcmkaKkC'
backup_model.predict(TheStand)

(array([1]), 0.968369207778855)

In [206]:
ClockworkAngel = 'rv5wCgAAQBAJ'
backup_model.predict(ClockworkAngel)

(array([1]), 0.9813968441378964)

In [209]:
BabyTeeth = 'Vuw7DwAAQBAJ'
backup_model.predict(BabyTeeth)

(array([1]), 0.999305999065374)

In [208]:
LookingForAlaska = '5v1NBhR1W88C'
backup_model.predict(LookingForAlaska)

(array([0]), 0.9652722735623045)

In [210]:
AmericanAssassin = 'kqgxDwAAQBAJ'
backup_model.predict(AmericanAssassin)

(array([1]), 0.924542898807493)

In [211]:
Grey = 'zS3ACQAAQBAJ'
backup_model.predict(Grey)

(array([1]), 0.7041480500189058)